# Step 1

* Load scalar, waveform and image h5 files.
* Snap the points onto a regular grid in x1-x2 space.
* Interpolate the points onto a regular y1 grid.
* Generate the file "rawgrid_....mmp" containing the 5D density array in slit-screen (x1-x2-y1-y3-x3) coordinates.
* Generate the file "rawgrid_coordinates_....npy" containing the mesh coordinates for "rawgrid_....mmp".

In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import imageio
from scipy import ndimage
from scipy import interpolate
import skimage
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from plotly import graph_objects as go
import proplot as pplt

sys.path.append('../..')
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

## Setup 

Global variables

In [ ]:
SMOOTH = False  # whether to smooth y1-y3-x3 image along first axis

Load info from step 0.

In [ ]:
folder = '_output'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
info

In [ ]:
datadir = info['datadir']
filename = info['filename']
file = h5py.File(join(datadir, 'preproc-' + filename + '.h5'), 'r')
data_sc = file['/scalardata']
data_wf = file['/wfdata']
data_im = file['/imagedata']

print('All attributes:')
print()
for data in [data_sc, data_wf, data_im]:
    print(data.name)
    for item in data.dtype.fields.items():
        print(item)
    print()

## Scan path

In [ ]:
variables = info['variables']
keys = list(variables)
ndim = len(keys)
acts = ['y_PositionSync', 'xp_PositionSync', 'x_PositionSync']
M = info['M']
Minv = np.linalg.inv(M)

center = np.array([variables[key]['center'] for key in keys])
distance = np.array([variables[key]['distance'] for key in keys])
nsteps = np.array([variables[key]['steps'] for key in keys])

points = np.vstack([data_sc[act] for act in acts]).T
points_n = utils.apply(Minv, points - center)
points_nn = points_n / (0.5 * distance)

We can compare these with the planned points listed in the csv file.

In [ ]:
csv_filename = join(datadir, filename + '.csv')
skiprows = 0
for line in open(csv_filename, 'r'):
    if line.startswith('#'):
        skiprows += 1
names = []
for i in range(1, 4):
    names.extend([f'start{i}', f'stop{i}', f'step{i}'])
df = pd.read_table(csv_filename, skiprows=skiprows, sep='\s+', header=None, names=names)
df

Make sure the actuator names in this notebook match the order in the CSV file.

In [ ]:
header_line = None
for row, line in enumerate(open(join(datadir, filename + '.csv'), 'r')):
    if row == skiprows - 1:
        header_line = line
        break
for char in ['#', ',']:
    header_line = header_line.replace(char, '')
header_line = header_line.lstrip()
print(header_line.split())

In [ ]:
# Extract planned actuator points.
pl_points = [df.loc[:, [f'start{i}', f'stop{i}']].values.ravel() 
             for i in range(1, ndim + 1)]
pl_points = np.array(pl_points).T

# Undo linear transformation.
pl_points_n = utils.apply(Minv, pl_points - center)
pl_points_nn = pl_points_n / (0.5 * distance)

# Plot planned points in "normalized" space.
fig, axes = pplt.subplots(nrows=3, figsize=(8.0, 4.0), spany=False, aligny=True)
for i, ax in enumerate(axes):
    ax.plot(pl_points_n[:, i], color='black', lw=None, marker='.', ms=0)
    ax.format(ylabel=f'actuator {i}')
plt.savefig('_output/planned_acts_n.png')

Create dataframe of planned actuator points in real and normalized space.

In [ ]:
dim_names = keys
columns = []
for i, dim_name in enumerate(dim_names):
    columns.extend([dim_name, dim_name + '_n', dim_name + '_nn'])
df = pd.DataFrame(index=df.index, columns=columns)
for i, dim_name in enumerate(dim_names):
    df[dim_name] = pl_points[::2, i]
    df[dim_name + '_n'] = pl_points_n[::2, i]
    df[dim_name + '_nn'] = pl_points_nn[::2, i]
df

In [ ]:
fig, ax = pplt.subplots()
ax.scatter(points_n[:, 1], points_n[:, 2], c='black', s=0.1, label='readback')
ax.scatter(pl_points_n[:, 1], pl_points_n[:, 2], c='red', s=0.1, label='planned')
ax.legend(loc='r', ncols=1, ms=1)
ax.format(xlabel='x2', ylabel='x1')
plt.savefig('_output/x2-x1_planned_readback.png')
plt.show()

## Interpolation 

### Snap points onto x1-x2 grid

We will use the planned points, which should be close to the real points in x1-x2. Need to play with `n` below until the expected number of grid points is obtained. We want the readback points to be relatively close to the grid points.

In [ ]:
ns_lists = []
for i in (1, 2):
    print('i =', i)
    diffs = []
    ns = np.arange(15, 500)
    for n in ns:
        gv, idx = utils.snap(pl_points_n[:, i], n=n, tol=0.1)
        deltas = np.diff(gv)
        diffs.append(np.var(deltas) / np.max(deltas))
    diffs = np.array(diffs)

    fig, ax = pplt.subplots(figsize=(5, 2))
    ax.plot(ns, diffs, color='black')
    from scipy.signal import find_peaks
    mins, _ = find_peaks(-diffs)
    ax.plot(ns[mins], diffs[mins], color='red', lw=0, marker='.', ms=3)
    plt.show()

    print(ns[mins][np.argsort(diffs[mins])])
    print()
    ns_lists.append(ns[mins][np.argsort(diffs[mins])])

In [ ]:
for n in range(100000):
    if n in ns_lists[0] and n in ns_lists[1]:
        print(n)

In [ ]:
gvs, idxs = [], []
for i, dim_name in enumerate(dim_names):
    gv, idx = utils.snap(
        pl_points_n[:, i], 
        n=97,  # 2022-04-09, 2021-12-03
    )    
    gvs.append(gv)
    idxs.append(idx)
    print(f'{dim_name}: {len(gv)} grid points')

In [ ]:
fig, axes = pplt.subplots(ncols=2, figwidth=9)
line_kws = dict(color='black', alpha=0.11)
for ax, title, pts in zip(axes, ['Planned', 'Readback'], [pl_points_n, points_n]):
    for gv in gvs[1]:
        ax.axvline(gv, **line_kws)
    for gv in gvs[2]:
        ax.axhline(gv, **line_kws)    
    ax.scatter(pts[:, 1], pts[:, 2], c='red', s=1)
    ax.format(title=title)
axes.format(xlabel='x2', ylabel='x1')
plt.savefig('_output/snap.png')

`POINTS2D` holds the iteration number on the x2-x1 grid. (The number of iterations is half the number of sweeps.)

In [ ]:
POINTS2D = np.full((len(gvs[2]), len(gvs[1])), np.nan)
iteration, steps = 0, []
for i in range(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        idx, = np.where((idxs[2] == i) & (idxs[1] == j))
        if len(idx) > 0:
            POINTS2D[i, j] = iteration
            steps.append(idx)
        else:
            print(f'Bin {i},{j} is empty')
        iteration += 1
POINTS2D = POINTS2D.astype(int)

In [ ]:
fig, ax = pplt.subplots()
_arr = np.zeros((POINTS2D.shape))
for i in range(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        _arr[i, j] = -1 if np.isnan(POINTS2D[i, j]) else POINTS2D[i, j]
ax.pcolormesh(
    gvs[1], 
    gvs[2],
    np.ma.masked_less_equal(_arr, 0),  cmap='grays',
    colorbar=True, colorbar_kw=dict(label='Iteration'),
)
ax.format(ylabel='x1', xlabel='x2')
plt.savefig('_output/POINTS2D.png')

`idx_bin[i]` holds a list of all indices for iteration `i` (the ith point in the x1-x2 grid).

In [ ]:
idx_bin = []
for i, step in enumerate(tqdm(steps)):
    iterations = np.unique((step / 2 + 1).astype(int))
    idx = np.hstack([np.where(data_sc[:, 'iteration'] == iteration)[0] for iteration in iterations])
    idx_bin.append(np.unique(idx))

### Observe one sweep

We will observe the sweep containing the largest camera integral in the scan.

In [ ]:
cam = info['cam']
ipeak = np.argmax(data_sc[cam + '_Integral'])
iteration_peak, n_iterations = None, len(steps)
for iteration in range(n_iterations):
    if ipeak in idx_bin[iteration]:
        iteration_peak = iteration
        
iteration = iteration_peak
idx = idx_bin[iteration]
print(f'Peak is in iteration {iteration}')
print(f'Set iteration = {iteration}')
print(f'Set idx = idx_bin[iteration] = {idx}')

Observe the actuator positions and camera integral during the sweep.

In [ ]:
bcm_mean = np.mean(data_sc[:, 'bcm04'])
bcm_scale = bcm_mean / data_sc[idx, 'bcm04']

kws = dict(marker='.', color='black')
fig, axes = pplt.subplots(nrows=4, figsize=(5, 6), spany=False, aligny=True)
for _scale, alpha, label in zip([1.0, bcm_scale], [1.0, 0.2], [None, 'Scaled by BCM04']):
    axes[0].plot(idx, data_sc[idx, cam + '_Integral'] * _scale,
                 alpha=alpha, label=label, **kws)
axes[0].legend(loc='upper right')
axes[0].format(ylabel=f'{cam}_Integral')
for ax, act in zip(axes[1:], acts):
    ax.plot(idx, data_sc[idx, act], **kws)
    ax.format(ylabel=act)
axes.format(xlabel='Step in scan', suptitle=f'Iteration {iteration}')
plt.savefig(f'_output/iteration{iteration}.png')
plt.show()

Observe the image on the screen during the sweep. (The following saves a GIF.)

In [ ]:
def get_image(i):
    image_shape = info['image_shape']
    return data_im[i, cam + '_Image'].reshape(image_shape)

In [ ]:
ipeak_sweep = np.argmax(data_sc[idx, cam + '_Integral'])
norm_pixel_value = np.max(data_im[idx[ipeak_sweep], cam + '_Image'])
for cmap in ['viridis', 'dusk_r', 'mono_r']:
    _cmap = pplt.Colormap(cmap)
    greyscale = False

    _ims = []
    for i in tqdm(idx):
        _im = get_image(i) / norm_pixel_value
        if not greyscale:
            _im = _cmap(_im)
        _ims.append(np.uint8(_im * np.iinfo(np.uint8).max))

    gif_filename = f'_output/iteration{iteration}_{cmap}.gif'
    imageio.mimwrite(gif_filename, _ims, fps=6)

### Form 3D image

Each sweep/iteration — each point in the x1-x2 grid — produces a 3D image: y1-y3-x3.

In [ ]:
im3d = np.array([get_image(i) for i in idx])
print(im3d.shape)

The image will be interpolated over the y1 axis. There is the option to apply a smoothing filter before interpolation. (I have actually found this to decrease the quality of the interpolation, but maybe that is because my images were already downscaled. This might have to do with the fact that I downscaled already? The variable `SMOOTH`, at the top of this notebook, determines whether smoothing is applied before interpolation.)

In [ ]:
im3d_smooth = ndimage.median_filter(im3d, size=(3, 1, 1), mode='constant', cval=0.0) 

Compare the image with/without smoothing at the frame with the largest pixel value in the iteration.

In [ ]:
i, _, _ = np.unravel_index(np.argmax(im3d), im3d.shape)

fig, axes = pplt.subplots([[1, 2, 3], [1, 2, 4]], figwidth=7, sharey=False)
axes[0].pcolormesh(im3d[i, :, :])
axes[1].pcolormesh(im3d_smooth[i, :, :])
for yscale in [None, 'log']:
    for ax in axes[2:]:
        handles = []
        for color, _im3d in zip(['black', 'pink6'], [im3d, im3d_smooth]):
            handle = ax.plot(np.sum(_im3d[i, :, :], axis=0), color=color, marker='.', ms=2, lw=1)
            handles.append(handle)
axes[3].format(yscale='log')
for ax, title in zip(axes, ['Raw', 'Smoothed along y', 'Profiles']):
    ax.set_title(title)
axes[:2].format(xticks=[], yticks=[])
axes[2].legend(handles, labels=['raw', 'smoothed'], ncol=1, loc='upper right', handlelength=1.25)
plt.savefig('_output/raw_vs_smooth_proj.png')
plt.show()

Look at different x3-y3 pixels for all y1.

In [ ]:
n = 7
ii = np.linspace(0, im3d.shape[1] - 1, n).astype(int)
jj = np.linspace(0, im3d.shape[2] - 1, n).astype(int)

fig, axes = pplt.subplots(ncols=n, nrows=n, figsize=(8.0, 5.5), space=0)
axes.format(
    xlabel='y1', 
    bottomlabels=['', '', '', 'x1', '', '', ''], 
    leftlabels=['', '', '', 'x2', '', '', ''],
)
for row, i in enumerate(ii):
    for col, j in enumerate(jj):
        ax = axes[row, col]
        kws = dict(lw=1.0)
        ax.plot(im3d[:, i, j], **kws)
        ax.plot(im3d_smooth[:, i, j], **kws)
plt.savefig('_output/y1_smoothing.png')
plt.show()

### Interpolate over y1

In [ ]:
# Define the y1 grid using all points in the data set.
N_PTS_Y = variables['y1']['steps']
y1_gv = np.linspace(
    np.min(data_sc['y_PositionSync']),
    np.max(data_sc['y_PositionSync']),
    N_PTS_Y,
)

# Obtain the y1 values corresponding to the y1 axis.
y1_vals = data_sc[idx, 'y_PositionSync']

# Interpolate along the y1 axis.
interp_method = 'linear'
_im3d = im3d_smooth if SMOOTH else im3d
arr3d = ip.interp_along_axis(_im3d, y1_vals, y1_gv, axis=0, kind=interp_method)

Find the the indices of the maximum element of the 3D image. Keep in mind that if `im3d` has many more y elements than `arr3d`, the plots may not look the same. 

In [ ]:
frame1, row, col = utils.max_indices(im3d)
frame2, _, _ = utils.max_indices(arr3d)

In [ ]:
fig, axes = pplt.subplots(ncols=2, nrows=2, sharey=False, figsize=(6, 4))
for x, arr, alpha, label in zip([y1_gv, y1_vals], [arr3d, im3d], [1.0, 0.2], ['interpolated', 'raw']):
    for j in range(2):
        y = utils.project(arr, 0) if j == 0 else arr[:, row, col]
        y = y / np.sum(y)
        for i, ax in enumerate(axes[:, j]):
            ax.plot(x, y, color='black', label=label, marker='.', alpha=alpha, ms=5)
axes[0, 1].legend(loc='upper right', ncol=1,)
axes[1, :].format(yscale='log', xlabel='y1')
for ax, title in zip(axes[0, :], ['Sum over image', f'Pixel ({row}, {col})']):
    ax.format(title=title)
plt.savefig('_output/peak_frame_interp_compare.png')

Plot the frame with the maximum pixel in either array.

In [ ]:
fig, axes = pplt.subplots(ncols=2, nrows=2, figwidth=None, sharex=False, sharey=False)
for j, _im in enumerate([im3d[frame1, :, :], arr3d[frame2, :, :]]):
    for i, norm in enumerate([None, 'log']):
        mplt.plot_image(_im.T / np.max(_im), ax=axes[i, j], norm=norm, colorbar=True)
axes.format(
    leftlabels=['Normal scale', 'Log scale'],
    toplabels=['Raw image', 'Interpolated along y']
)
plt.savefig('_output/interp_image_compare.png')

In [ ]:
fig, axes = pplt.subplots(ncols=2, sharey=False, spanx=False, figsize=(8, 2))
axes[0].plot(np.sum(arr3d[frame2, :, :], axis=0), color='black', alpha=0.2, label='interpolated')
axes[0].plot(np.sum(im3d[frame1, :, :], axis=0), marker='.', ms=2, lw=0, color='black', label='raw')
axes[0].legend(ncol=1)
axes[0].format(yscale=None, title='ProfileX')
axes[1].plot(arr3d[-2, :, :].ravel(), marker='.', lw=0, color='black', ms=1, label='Raw')
axes[1].plot(arr3d[-2, :, :].ravel(), color='black', alpha=0.2, label='Interpolated')
axes[1].legend(loc='upper left', ncols=1)
axes[1].format(title='Pixels in edge frame', xlabel='Pixel number')
plt.savefig('_output/interp_px.png')

In [ ]:
fig, axes = pplt.subplots([[1, 2, 3], [1, 4, 5]], sharey=False, sharex=False)
axes[0].pcolormesh(im3d[frame1, :, :])
kws = dict(color='white', lw=0.785, alpha=0.5)
axes[0].axvline(col, **kws)
axes[0].axhline(row, **kws)
kws = dict(marker='.', color='black', ms=2.0, lw=1.0)
for im, label in zip([arr3d[frame2], im3d[frame1]], ['interpolated', 'raw']):
    if label == 'raw':
        kws['alpha'] = 0.2
    elif label == 'interpolated':
        kws['alpha'] = 1.0
    for ax in axes[:, 1]:
        ax.plot(im[row, :] / np.sum(im[row, :]), label=label, **kws)
    for ax in axes[:, 2]:
        ax.plot(im[:, col] / np.sum(im[:, col]), label=label, **kws)
axes[2].legend(ncols=1, loc='upper right')
axes[1].set_title(f'{cam} - row {row}')
axes[2].set_title(f'{cam} - col {col}')
axes[1, 1:].format(yscale='log')
plt.savefig('_output/interp_peak_slice.png')
plt.show()

Now try comparing all x3-y3 projections of 3D image — interpolated vs. not.

In [ ]:
ncols = 8
nplots = 3 * ncols
for j, (title, _arr) in enumerate(zip(['Raw', 'Interpolated along y1'], [im3d, arr3d])):
    ind = np.linspace(0, _arr.shape[0] - 1, 24).astype(int)
    nrows = np.ceil(len(ind) / float(ncols)).astype(int)
    fig, axes = pplt.subplots(ncols=ncols, nrows=nrows, figwidth=10.0, space=0.2)
    axes.format(xticks=[], yticks=[])
    _y1 = [y1_vals, y1_gv][j]
    for ax, i in zip(axes, ind):
        mplt.plot_image(_arr[i].T, ax=ax)
        ax.annotate(f'y1 = {_y1[i]:.2f}', xy=(0.02, 0.98), xycoords='axes fraction',
                    color='white', verticalalignment='top')
    for i in range(nrows * ncols - len(ind)):
        axes[~i].axis('off')
    axes.format(suptitle=title)
    plt.savefig(f"_output/iteration{iteration}_{['raw','y1interp'][j]}.png")
    plt.show()

## View projections and slices of interpolated 3D image 

### Projections

In [ ]:
dims = ['y1', 'y3', 'x3']
axes = mplt.corner(arr3d, diag_kind=None, labels=dims, prof='edges',
                   prof_kws=dict(lw=0.5, alpha=0.7))
plt.savefig('_output/peak_interp_proj_corner.png')

### Slices

In [ ]:
ind_slice = [int(s / 2) for s in arr3d.shape]

plot_kws = dict(discrete=False, colorbar=True)
fig, axes = pplt.subplots(nrows=2, ncols=3, sharex=False, sharey=False, spany=False, spanx=False, )
for j in range(3):
    idx = utils.make_slice(3, j, ind_slice[j])
    _im = arr3d[idx]
    _im = _im / np.max(_im)
    for ax, norm in zip(axes[:, j], [None, 'log']):
        mplt.plot_image(_im, ax=ax, norm=norm, **plot_kws)
    axes[0, j].format(title=f'{dims[j]} = {ind_slice[j]}')
axes[:, 0].format(xlabel='y3', ylabel='x3')
axes[:, 1].format(xlabel='y1', ylabel='x3')
axes[:, 2].format(xlabel='y1', ylabel='y3')
plt.savefig('_output/peak_interp_slice.png')
plt.show()

In [ ]:
mplt.interactive_proj2d(arr3d / np.max(arr3d), dims=['x1', 'y3', 'x3'], 
                        default_ind=(2, 1), slider_type='int')

## Save 3D grid coordinates

In [ ]:
GV_X1_n, GV_X2_n = np.meshgrid(gvs[2], gvs[1], indexing='ij')
x2_x1_n = np.vstack([GV_X2_n.ravel(), GV_X1_n.ravel()]).T
x2_x1 = utils.apply(M[1:, 1:], x2_x1_n) + center[1:]
GV_X2 = x2_x1[:, 0].reshape(GV_X2_n.shape)
GV_X1 = x2_x1[:, 1].reshape(GV_X1_n.shape)

fig, axes = pplt.subplots(ncols=2, figwidth=8, sharey=False, sharex=False)
for ax, (grid1, grid2), pts in zip(axes, [(GV_X2, GV_X1), (GV_X2_n, GV_X1_n)], [pl_points, pl_points_n]):
    _h1 = ax.plot(grid1.T, grid2.T, color='black', alpha=0.2, lw=3)
    _h2 = ax.plot(pts[:, 1], pts[:, 2], color='black', marker='.', ms=2, lw=0.5)
axes.format(xlabel='x2', ylabel='x1')
axes[0].legend([_h1, _h2], labels=['grid', 'scan rb'], loc='upper left')
axes[1].format(title='Normalized')
plt.savefig('_output/x1-x2-grid.png')

Obtain 3D grid coordinates.

In [ ]:
X1 = np.repeat(GV_X1[:, :, np.newaxis], N_PTS_Y, axis=2)
X2 = np.repeat(GV_X2[:, :, np.newaxis], N_PTS_Y, axis=2)
Y1 = np.zeros(X1.shape)
for i in range(Y1.shape[0]):
    for j in range(Y1.shape[1]):
        Y1[i, j, :] = y1_gv

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(5, 4), spany=False, aligny=True)
for ax, G, ylabel in zip(axes, [X1, X2, Y1], ["x1 [mm]", "x2 [mm]", "y1 [mm]"]):
    ax.plot(G.ravel(), color='black')
    ax.format(ylabel=ylabel)
axes.format(xlabel='Step')
plt.savefig('_output/acts_black.png')

In [ ]:
coord_3d = np.stack([X1, X2, Y1], axis=0)
print(f'coord_3d.shape = {coord_3d.shape}')

In [ ]:
savefilename = f'_output/coordinates3d_raw_{filename}.npy'
np.save(savefilename, coord_3d)

## Save 5D array as memory map 

In [ ]:
shape = [len(gvs[2]), len(gvs[1])] + list(arr3d.shape)  # [x1, x2, y1, y3, x3]
shape = tuple(shape)
info['rawgrid_shape'] = shape
print('shape:', shape)

In [ ]:
im_dtype = get_image(0).dtype
info['im_dtype'] = im_dtype
print('image dtype:', im_dtype)

In [ ]:
utils.save_pickle('_output/info.pkl', info)
file = open('_output/info.txt', 'w')
for key, value in info.items():
    file.write(f'{key}: {value}\n')
file.close()

In [ ]:
savefilename = f'_output/f_raw_{filename}.mmp'
f_raw = np.memmap(savefilename, shape=shape, dtype=im_dtype, mode='w+') 
for i in trange(POINTS2D.shape[0]):
    for j in range(POINTS2D.shape[1]):
        try:
            iteration = POINTS2D[i, j]
            idx = idx_bin[iteration]
            im3d = np.array([get_image(k) for k in idx])
            if SMOOTH:
                im3d = ndimage.median_filter(im3d, size=(3, 1, 1), mode='constant', cval=0.0) 
            y1_vals = data_sc[idx, 'y_PositionSync']
            f_raw[i, j, :, :, :] = ip.interp_along_axis(im3d, y1_vals, y1_gv, axis=0, kind=interp_method)
        except IndexError:
            print(f'No points in grid bin ({i}, {j})')
            f_raw[i, j, :, :, :] = 0
del f_raw

In [ ]:
f_raw = np.memmap(savefilename, shape=shape, dtype=im_dtype) 